In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor()]
)

train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [8]:
class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.layer1 = nn.Linear(28*28, 128) # Input-Layer (28x28) -> Hidden Layer 128
        self.layer2 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.2) # Vermeidung vom Overfitting
        self.layer3 = nn.Linear(64,10) # Output -> Wahrscheinlichkeiten für Produktkategorien

    def forward(self,x):
        x = x.view(-1, 28*28) # (2D -> 1D), Flatten der Eingabe
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.dropout(x) # Dropout anwenden
        x = self.layer3(x) # Output-Layer
        return x 


In [9]:
model = NN()

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)   # moderne Variante von SGD

num_epochs = 20 # 20 mal werden wir die Daten unserem Algorithmus zeigen

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images,labels in train_loader: # Samplen aus Trainingsdaten 
        optimizer.zero_grad() # Gradienten zurücksetzen
        outputs = model(images) 
        curr_loss = loss(outputs, labels)

        curr_loss.backward() # Ableitung
        optimizer.step()

        running_loss += curr_loss.item()
    
    print("Epoch [" + str(epoch+1) + "/" + str(num_epochs) + "], Loss:" + str(running_loss))


Epoch [1/20], Loss:564.7575409263372
Epoch [2/20], Loss:385.0481908470392
Epoch [3/20], Loss:347.2964168712497
Epoch [4/20], Loss:321.6767787337303
Epoch [5/20], Loss:305.29730136692524
Epoch [6/20], Loss:290.8224961683154
Epoch [7/20], Loss:276.81886015087366
Epoch [8/20], Loss:266.6976313665509
Epoch [9/20], Loss:258.8212715908885
Epoch [10/20], Loss:250.42174124717712
Epoch [11/20], Loss:242.5419107005
Epoch [12/20], Loss:235.29350101202726
Epoch [13/20], Loss:229.10020893067122
Epoch [14/20], Loss:223.53712476044893
Epoch [15/20], Loss:220.13756193593144
Epoch [16/20], Loss:212.74210831150413
Epoch [17/20], Loss:206.97866959869862
Epoch [18/20], Loss:203.1010755673051
Epoch [19/20], Loss:196.5588060952723
Epoch [20/20], Loss:193.94845896586776


In [10]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        predicted = torch.max(outputs.data,1)[-1] # argmax von outputs = die Kategorie mit der maximalen Wahrscheinlichkeit
        total += labels.size(0) # Gesamtanzahl
        correct += (predicted==labels).sum().item() # Anzahl der korrekt klassifizierten Labels

accuracy = correct/total
print("Genauigkeit: " + str(accuracy))

Genauigkeit: 0.8905


In [7]:
total

10000